In [1]:
%matplotlib inline
from skimage import * # utils like img_as_ubyte
from skimage import io, filters, morphology
from skimage.io import imread, imsave, imshow
from skimage.transform import resize

from scipy import ndimage

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.pyplot import figure, subplot, plot, yticks, xticks
from matplotlib.pyplot import legend, colorbar, xlim, ylim, xlabel, ylabel
# projection='3d'
from mpl_toolkits.mplot3d.axes3d import Axes3D

from leicaexperiment import Experiment, attributes
from leicascanningtemplate import ScanningTemplate

from microscopestitching import stitch

from glob import glob

from joblib import Parallel, delayed

from math import ceil
import numpy as np

# matplotlib

In [ ]:
# style - http://www.huyng.com/posts/sane-color-scheme-for-matplotlib/
plt.style.use('ggplot')

In [ ]:
def imshow_range(img, **kwargs):
    "Show only range of intensities in image"
    imgplot = io.imshow(img, vmin=img.min(), vmax=img.max(), **kwargs)
    colorbar()
    return imgplot

In [ ]:
def imshow_simple(img, **kwargs):
    imgplot = io.imshow(img, **kwargs)
    no_ticks(imgplot.axes)
    return imgplot

In [3]:
def imshow_downscale(img):
    "matplotlib is slooooow, showing downscaled images is speedier"
    if type(img) == str:
        img = imread(img)
    y,x = img.shape
    # downscale factor
    k = ceil(y//512+0.1)
    imgplot = imshow(resize(img, (y//k, x//k)))

    # real ticks
    if y > x:
        # less ticks if not square image
        y_step = y//6
        x_step = int(y_step*x/y)
    if x < y:
        y_step = y//6
        x_step = int(y_step*y/x)
    else:
        y_step = y//6
        x_step = x//6

    yticks(range(0,y//k,y_step//k), [str(y) for y in range(0,y,y_step)])
    xticks(range(0,x//k,x_step//k), [str(x) for x in range(0,x,x_step)])
    
    return imgplot

In [4]:
def side_by_side(*images):
    "view images side by side"
    size = len(images)
    figure(figsize=(size*5,5))
    for i,img in enumerate(images):
        subplot(1, size, i+1)
        imshow_downscale(img)

In [6]:
def image_axis(ax, ticks=None):
    "Sets tick outward and x ticks on the top"
    ax.xaxis.set_label_position('top') 
    ax.xaxis.tick_top()
    ax.xaxis.set_tick_params(direction='out')

    ax.yaxis.tick_left()
    ax.yaxis.set_tick_params(direction='out')
    
    if ticks:
        ax.yaxis.set_ticks(ticks)
        ax.xaxis.set_ticks(ticks)

In [6]:
def no_ticks(ax):
    "turns of ticks in axes"
    ax.yaxis.set_ticks([])
    ax.xaxis.set_ticks([])

In [ ]:
# ranges for images
def range_img(n):
    "for use with mpl ticks. n = img size"
    return range(0, n+1, (n+1)//4)

In [4]:
def plot_intensities(*imgs):
    if len(imgs) > 1:
        figure(figsize=(len(imgs)*4,3))
    for n, img in enumerate(imgs):
        if type(img) == str:
            img = imread(img)
        subplot(1,len(imgs),n+1)
        for line in img:
            plot(line, 'b.')
        xlim(0, img.shape[1])

# remove intensity variation

In [5]:
def remove_intensity_variation(img, intensity_profile, intensity_range=None):
    """Equalizes rows in image according to intensity profile.
    
    Parameters
    ----------
    img : 2d array or string
        Image to equalize.
    intensity_profile : array
        Normalized profile of intensity.
    intensity_range : (min,max) int
        Normalize to this range before equalizing.
    """
    filename = False
    if type(img) == str:
        filename = img
        img = imread(img).astype(np.float)
    else:
        img = img.astype(np.float)
    
    if not intensity_range:
        intensity_range = img.min(), img.max()
    
    # normalize
    img -= intensity_range[0]
    img /= intensity_range[1] - intensity_range[0]
    
    # equalize
    img /= intensity_profile

    # clip values
    img[img < 0] = 0 # there should be no values below 0
    img[img > 1] = 1
    img *= 255
    img = img.astype(np.uint8)
    
    if filename:
        imsave(filename, img)
    else:
        return img

In [1]:
def equalize_experiment(experiment):
    """Remove uneven illumination from leica experiment. Experiment is
    copied before equalized.
    
    Returns
    -------
    leicaexperiment.Experiment
    """
    # copy experiment
    from shutil import copytree
    new_name = experiment.path + '-equalized'
    copytree(experiment.path, new_name)
    e = Experiment(new_name)
    
    # find line with least variance
    p1 = e.image(0,0,0,0)
    img = imread(p1)
    line_variance = np.var(img, axis=0)
    line = np.argmin(line_variance)
    background_row = np.copy(img[line,:])

    # curve fit
    from scipy.optimize import curve_fit

    width = img.shape[1]
    x = np.arange(width)
    
    def polynomial(x,a,b,c):
        return a*x**2+b*x+c

    coefficients, covariance = curve_fit(polynomial, x, background_row)
    a,b,c = coefficients
    y = a*x**2+b*x+c
    intensity_profile = y / y.max()
    
    # filter images
    # equalize eq_e experiment
    def min_max(filename):
        img = imread(filename)
        return img.min(), img.max()

    mm = Parallel(n_jobs=4)(delayed(min_max)(f) for f in e.images)
    mm = np.array(mm)
    intensity_range = np.median(mm[:, 0]), np.median(mm[:, 1])

    Parallel(n_jobs=4)(delayed(remove_intensity_variation)
          (f, intensity_profile, intensity_range) for f in e.images)
    
    return e

# image collection

In [1]:
def show_all_images(images):
    from skimage import viewer
    if type(images[0]) == str:
        images = [imread(i) for i in images]
    v = viewer.viewers.CollectionViewer(images)
    v.show()

/Users/arve/.virtualenvs/3.4/lib/python3.4/site-packages/skimage/viewer/utils/core.py:11: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warnings.warn("Recommended matplotlib backend is `Agg` for full "


# export

In [27]:
MARGIN = 5 # mm
B5 = (250, 176) # y,x mm
B5_HEIGTH = (B5[0] - 2*MARGIN) / 25.4 # inches
B5_WIDTH = (B5[1] - 2*MARGIN) / 25.4 # inches

def savefig(filename, **kwargs):
    "short hand to prevent 600dpi in notebooks"
    fig = plt.gcf()
    
    x,y = fig.get_size_inches()
    scale = B5_WIDTH / x
    size = 2*B5_WIDTH, 2*y*scale
    fig.set_size_inches(size)
    
    plt.tight_layout()
    plt.savefig(filename, dpi=600, **kwargs)

    fig.set_size_inches((x,y))

# stitch

In [5]:
def merge(img1, img2, dy, dx):
    y,x = img1.shape
    yy,xx = img2.shape
    
    if dy >= 0:
        y_pos1 = slice(0, y)
        y_pos2 = slice(dy, yy+dy)
    if dy < 0:
        y_pos1 = slice(-dy, y-dy)
        y_pos2 = slice(0, yy)
    if dx >= 0:
        x_pos1 = slice(0, x)
        x_pos2 = slice(dx, xx+dx)
    if dx < 0:
        x_pos1 = slice(-dx, x-dx)
        x_pos2 = slice(0, xx)
    
    Y = max((y_pos1.stop, y_pos2.stop)) - min((y_pos1.start, y_pos2.start))
    X = max((x_pos1.stop, x_pos2.stop)) - min((x_pos1.start, x_pos2.start))
    
    merged = np.zeros((Y,X), dtype=np.int)
    merged[y_pos1,x_pos1] += img1
    merged[y_pos2,x_pos2] += img2
    
    # average the overlap
    overlap = (slice(dy, y), slice(dx, x))
    merged[overlap] //= 2
    
    return merged.astype(np.uint8)

# pretty xml

In [1]:
import xml.dom.minidom as minidom

def pretty_xml(filename):
    dom = minidom.parse('{ScanningTemplate}asdf.lrp')
    return dom.toprettyxml('  ')

# zick zack sort

In [38]:
from operator import attrgetter

def zick_zack_sort(list_, sortby):
    if type(sortby) is str:
        sortby = (sortby,)

    list_.sort(key=attrgetter(*sortby))
    
    firstgetter = attrgetter(sortby[0])
    prev = firstgetter(list_[0])

    out = []
    revert = False
    for i in list_:
        cur = firstgetter(i)
        if not revert and cur != prev:
            revert = True
            part = []
        elif revert and cur != prev:
            out.extend(part[::-1])
            revert = False
        #print(i, revert, cur, prev)
        if revert:
            part.append(i)
        if not revert:
            out.append(i)
        prev = cur
    return out

In [41]:
#class R:
#    def __init__(self, x, y):
#       self.x = x
#        self.y = y
#    def __repr__(self):
#        return "R(%s,%s)" % (self.x, self.y)
#
#regions = [R(1,1), R(1,2), R(1,3), R(2,1), R(2,2), R(2,3), R(3,1), R(3,2), R(3,3)]
#zick_zack_sort(regions, ('y', 'x'))

[R(1,1), R(2,1), R(3,1), R(3,2), R(2,2), R(1,2), R(1,3), R(2,3), R(3,3)]

In [42]:
#zick_zack_sort(regions, ('x', 'y'))

[R(1,1), R(1,2), R(1,3), R(2,3), R(2,2), R(2,1), R(3,1), R(3,2), R(3,3)]